In [2]:
import urllib.request as ur
import re
import pandas as pd


In [3]:
samples = ['J000092712', 'J000106560', 'TM00016', 'TM00065', 'TM00089', 'TM00090', 'TM00090', 'TM00096', 'TM00107', 'TM00137', 'TM00170', 'TM00226', 'TM00231', 'TM00256', 'TM00279', 'TM00314', 'TM00335', 'TM00999', 'TM01571']
#samples = ['J000092712', 'TM00090']

In [66]:
jax_strna = pd.DataFrame({'Gene': []}).set_index('Gene')
jax_rna = pd.DataFrame({'Gene': []}).set_index('Gene')

for sample in samples:
    page = ur.urlopen(f'http://tumor.informatics.jax.org/mtbwi/pdxDetails.do?modelID={sample}').read().split(b'\r\n')
    for line in page:
        data,count = re.subn(b'\s*expressionData = google.visualization.arrayToDataTable\((.*)\);\s*',b'\g<1>',line)
        if count:
            data = re.sub(b'true',b'True',data)
            data = re.sub(b'false',b'False',data)
            data = re.sub(b"{role:'certainty'}",b'None',data)
            data = re.sub(b'null',b'None',data)
            data = eval(data)
            expr_data = pd.DataFrame(data[1:],columns=data[0]).set_index('Gene')
            counts = expr_data.filter(like='stRNA_Seq')
            if not counts.columns.empty:
                counts.columns=[sample]
                jax_strna = pd.concat([jax_strna, counts],axis = 1)
            else:
                counts = expr_data.filter(like=' RNA_Seq')
                counts.columns=[sample]
                jax_rna = pd.concat([jax_rna,counts],axis = 1) 

In [17]:
meansd_rna = pd.read_csv('rnaseq-meansd.csv',delimiter=';').rename(columns={'gene_symbol_hint':'Gene'}).set_index('Gene').filter(items=['mean','sd'])
meansd_strna = pd.read_csv('strnaseq-meansd.csv',delimiter=';').rename(columns={'gene_symbol_hint':'Gene'}).set_index('Gene').filter(items=['mean','sd'])

In [70]:
meansd_rna = meansd_rna[meansd_rna.index.isin(jax_rna.index)]
jax_rna = jax_rna[jax_rna.index.isin(meansd_rna.index)]

meansd_strna = meansd_strna[meansd_strna.index.isin(jax_strna.index)]
jax_strna = jax_strna[jax_strna.index.isin(meansd_strna.index)]

In [95]:
jax_expr = pd.concat([jax_rna.mul(meansd_rna['sd'],axis='index').add(meansd_rna['mean'],axis='index'),
    jax_strna.mul(meansd_strna['sd'],axis='index').add(meansd_strna['mean'],axis='index')], axis=1)